# College Football Database
Taylor Lehman  
D532 Applied Database Technologies

In [18]:
import pandas as pd
import numpy as np
import psycopg2

In [62]:
# Set up connection
conn = psycopg2.connect(
    dbname="bsb_db",
    user="postgres",
    password="ds865aD5",
    host="localhost",
    port="5432"
)

cur = conn.cursor()

print("Connected to Postres database: bsb_db")

Connected to Postres database: bsb_db


### Create Database to Connect on PostgreSQL Above
This is just initially establishing the database, using the default db "postgres" so that I can connect to my database "bsb_db" easily.  
Requires backtracking but will ensure easier connections after creation.

In [9]:
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="ds865aD5",
    host="localhost",
    port="5432"
)

cur = conn.cursor()
conn.autocommit = True

try:
    cur.execute("CREATE DATABASE bsb_db;")
    print("Database bsb_db created!")
except psycopg2.errors.DuplicateDatabase:
    print("Database bsb_db already exists.")

conn.autocommit = False
cur.close()
conn.close()

Database bsb_db created!


### Reading/Writing SQL
The SQL code below is used to create/update the schema of the database.  
There is a cell underneath to simply read the .sql file if need be.

In [103]:
schema_sql = """
-- Drop tables if they exist (in reverse FK order)
DROP TABLE IF EXISTS success, epa, pff_defense, pff_blocking, pff_receiving, pff_rushing, pff_passing, plays, drives, games, teams CASCADE;

-- TEAMS
CREATE TABLE teams (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100) NOT NULL,
    conference VARCHAR(50),
    city VARCHAR(100),
    state VARCHAR(50)
);

-- GAMES
CREATE TABLE games (
    game_id BIGINT PRIMARY KEY,
    year INTEGER NOT NULL,
    week INTEGER NOT NULL,
    home_team_id INTEGER NOT NULL,
    home VARCHAR(100),
    away_team_id INTEGER NOT NULL,
    away VARCHAR(100),
    home_score INTEGER,
    away_score INTEGER,
    FOREIGN KEY (home_team_id) REFERENCES teams(team_id),
    FOREIGN KEY (away_team_id) REFERENCES teams(team_id)
);

-- DRIVES
CREATE TABLE drives (
    drive_id BIGINT PRIMARY KEY,
    game_id BIGINT NOT NULL,
    drive_num INTEGER,
    pos_team_id INTEGER,
    pos_team VARCHAR(100),
    drive_result VARCHAR(100),
    drive_yards INTEGER,
    drive_start_yards_to_goal INTEGER,
    drive_end_yards_to_goal INTEGER,
    drive_points INTEGER,
    drive_time_minutes_start INTEGER,
    drive_time_seconds_start INTEGER,
    drive_time_minutes_end INTEGER,
    drive_time_seconds_end INTEGER,
    drive_time_minutes_elapsed INTEGER,
    drive_time_seconds_elapsed INTEGER,
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (pos_team_id) REFERENCES teams(team_id)
);

-- PLAYS
CREATE TABLE plays (
    id_play INTEGER PRIMARY KEY,
    game_id BIGINT NOT NULL,
    game_play_number INTEGER,
    drive_id BIGINT,
    half INTEGER,
    period INTEGER,
    clock_minutes INTEGER,
    clock_seconds INTEGER,
    time_secs_rem INTEGER,
    pos_team VARCHAR(100),
    pos_team_id INTEGER,
    def_pos_team VARCHAR(100),
    def_pos_team_id INTEGER,
    play_type VARCHAR(100),
    play_text TEXT,
    pass INTEGER,
    pass_attempt INTEGER,
    completion INTEGER,
    rush INTEGER,
    yards_gained INTEGER,
    down INTEGER,
    distance INTEGER,
    yards_to_goal INTEGER,
    epa FLOAT,
    def_epa FLOAT,
    wpa FLOAT,
    ep_before FLOAT,
    ep_after FLOAT,
    wp_before FLOAT,
    wp_after FLOAT,
    sack INTEGER,
    int INTEGER,
    penalty_detail VARCHAR(255),
    yds_penalty INTEGER,
    firstd_by_penalty INTEGER,
    firstd_by_yards INTEGER,
    success INTEGER,
    turnover INTEGER,
    turnover_downs INTEGER,
    score_pts INTEGER,
    scoring_play INTEGER,
    defense_score_play INTEGER,
    change_of_poss INTEGER,
    off_timeout INTEGER,
    off_timeouts_rem_before INTEGER,
    def_timeout INTEGER,
    def_timeouts_rem_before INTEGER,
    FOREIGN KEY (game_id) REFERENCES games(game_id),
    FOREIGN KEY (drive_id) REFERENCES drives(drive_id),
    FOREIGN KEY (pos_team_id) REFERENCES teams(team_id),
    FOREIGN KEY (def_pos_team_id) REFERENCES teams(team_id)
);

-- PASSING
CREATE TABLE pff_passing (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100),
    attempts INTEGER,
    avg_depth_of_target FLOAT,
    avg_time_to_throw FLOAT,
    big_time_throws INTEGER,
    completion_percentage FLOAT,
    completions INTEGER,
    def_gen_pressures INTEGER,
    dropbacks INTEGER,
    drops INTEGER,
    grade_offense FLOAT,
    grade_passing FLOAT,
    grade_run FLOAT,
    hit_as_threw INTEGER,
    interceptions INTEGER,
    sacks INTEGER,
    scrambles INTEGER,
    thrown_away INTEGER,
    touchdowns INTEGER,
    turnover_worthy_plays INTEGER,
    yards INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- RUSHING
CREATE TABLE pff_rushing (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100),
    attempts INTEGER,
    avoided_tackles INTEGER,
    breakaway_yards INTEGER,
    first_downs INTEGER,
    fumbles INTEGER,
    grades_offense FLOAT,
    grades_pass_block FLOAT,
    grades_run FLOAT,
    rec_yards INTEGER,
    receptions INTEGER,
    routes INTEGER,
    scramble_yards INTEGER,
    targets INTEGER,
    touchdowns INTEGER,
    yards INTEGER,
    yards_after_contact INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- RECEIVING
CREATE TABLE pff_receiving (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100),
    contested_receptions INTEGER,
    contested_targets INTEGER,
    grades_offense FLOAT,
    grades_pass_route FLOAT,
    receptions INTEGER,
    routes INTEGER,
    yards INTEGER,
    yards_after_catch INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- BLOCKING
CREATE TABLE pff_blocking (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100),
    grades_offense FLOAT,
    grades_pass_block FLOAT,
    grades_run_block FLOAT,
    hits_allowed INTEGER,
    hurries_allowed INTEGER,
    penalties INTEGER,
    pressures_allowed INTEGER,
    sacks_allowed INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- DEFENSE
CREATE TABLE pff_defense (
    team_id INTEGER PRIMARY KEY,
    team_name VARCHAR(100),
    batted_passes INTEGER,
    forced_fumbles INTEGER,
    fumble_recoveries INTEGER,
    grades_coverage_defense FLOAT,
    grades_defense FLOAT,
    grades_pass_rush_defense FLOAT,
    grades_run_defense FLOAT,
    grades_tackle FLOAT,
    hits INTEGER,
    hurries INTEGER,
    interceptions INTEGER,
    missed_tackles INTEGER,
    pass_break_ups INTEGER,
    penalties INTEGER,
    qb_rating_against FLOAT,
    receptions INTEGER,
    sacks FLOAT,
    safeties INTEGER,
    stops INTEGER,
    tackles INTEGER,
    tackles_for_loss FLOAT,
    total_pressures INTEGER,
    touchdowns INTEGER,
    yards INTEGER,
    yards_after_catch INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- EPA
CREATE TABLE epa (
    team_id INTEGER PRIMARY KEY,
    epa_per_play FLOAT,
    def_epa_per_play FLOAT,
    total_epa FLOAT,
    def_total_epa FLOAT,
    passing_epa_per_play FLOAT,
    def_passing_epa_per_play FLOAT,
    passing_total_epa FLOAT,
    def_passing_epa_total FLOAT,
    rushing_epa_per_play FLOAT,
    def_rushing_epa_per_play FLOAT,
    rushing_total_epa FLOAT,
    def_rushing_total_epa FLOAT,
    firstdown_epa_per_play FLOAT,
    def_firstdown_epa_per_play FLOAT,
    seconddown_epa_per_play FLOAT,
    def_seconddown_epa_per_play FLOAT,
    thirddown_epa_per_play FLOAT,
    def_thirddown_epa_per_play FLOAT,
    fourthdown_epa_per_play FLOAT,
    def_fourthdown_epa_per_play FLOAT,
    redzone_epa_per_play FLOAT,
    def_redzone_epa_per_play FLOAT,
    redzone_total_epa FLOAT,
    def_redzone_total_epa FLOAT,
    past40_epa_per_play FLOAT,
    def_past40_epa_per_play FLOAT,
    past40_total_epa FLOAT,
    def_past40_total_epa FLOAT,
    total_penalty_epa FLOAT,
    def_total_penalty_epa FLOAT,
    timeout_total_epa FLOAT,
    def_timeout_total_epa FLOAT,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);

-- SUCCESS
CREATE TABLE success (
    team_id INTEGER PRIMARY KEY,
    success_rate FLOAT,
    def_success_rate FLOAT,
    passing_success_rate FLOAT,
    def_passing_success_rate FLOAT,
    rushing_success_rate FLOAT,
    def_rushing_success_rate FLOAT,
    firstdown_success_rate FLOAT,
    def_firstdown_success_rate FLOAT,
    seconddown_success_rate FLOAT,
    def_seconddown_success_rate FLOAT,
    thirddown_success_rate FLOAT,
    def_thirddown_success_rate FLOAT,
    fourthdown_success_rate FLOAT,
    def_fourthdown_success_rate FLOAT,
    redzone_success_rate FLOAT,
    def_redzone_success_rate FLOAT,
    past40_success_rate FLOAT,
    def_past40_success_rate FLOAT,
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
);
"""

In [104]:
# Write schema to file
with open("football_data_schema.sql", "w") as schema:
    schema.write(schema_sql)

print("SQL file updated.")

SQL file updated.


In [106]:
# Read sql file 
try:
    with open("football_data_schema.sql", "r") as schema:
        sql = schema.read()
    cur.execute(sql)
    conn.commit()
    print("Schema successfully loaded into PostgreSQL.")
except Exception as e:
    conn.rollback()
    print("Error loading schema:", e)

Schema successfully loaded into PostgreSQL.


# Maps and Lists for Manipulating Team Info

In [22]:
fbs_teams = [
    "Air Force", "Akron", "Alabama", "Appalachian State", "Arizona",
    "Arizona State", "Arkansas", "Arkansas State", "Army", "Auburn",
    "Ball State", "Baylor", "Boise State", "Boston College", "Bowling Green",
    "Buffalo", "BYU", "California", "Central Michigan", "Charlotte",
    "Cincinnati", "Clemson", "Coastal Carolina", "Colorado", "Colorado State",
    "Connecticut", "Duke", "East Carolina", "Eastern Michigan", "Florida",
    "Florida Atlantic", "Florida International", "Florida State", "Fresno State",
    "Georgia", "Georgia Southern", "Georgia State", "Georgia Tech", "Hawai'i",
    "Houston", "Illinois", "Indiana", "Iowa", "Iowa State",
    "Jacksonville State", "James Madison", "Kansas", "Kansas State", "Kennesaw State",
    "Kent State", "Kentucky", "Liberty", "Louisiana", "Louisiana Monroe",
    "Louisiana Tech", "Louisville", "LSU", "Marshall", "Maryland",
    "Memphis", "Miami", "Miami (OH)", "Michigan", "Michigan State",
    "Middle Tennessee", "Minnesota", "Mississippi State", "Missouri", "Navy",
    "NC State", "Nebraska", "Nevada", "New Mexico", "New Mexico State",
    "North Carolina", "North Texas", "Northern Illinois", "Northwestern", "Notre Dame",
    "Ohio", "Ohio State", "Oklahoma", "Oklahoma State", "Old Dominion",
    "Ole Miss", "Oregon", "Oregon State", "Penn State", "Pittsburgh",
    "Purdue", "Rice", "Rutgers", "Sam Houston State", "San Diego State",
    "San José State", "SMU", "South Alabama", "South Carolina", "South Florida",
    "Southern Mississippi", "Stanford", "Syracuse", "TCU", "Temple",
    "Tennessee", "Texas", "Texas A&M", "Texas State", "Texas Tech",
    "Toledo", "Troy", "Tulane", "Tulsa", "UAB",
    "UCF", "UCLA", "UMass", "UNLV", "USC",
    "UTEP", "UT San Antonio", "Utah", "Utah State", "Vanderbilt",
    "Virginia", "Virginia Tech", "Wake Forest", "Washington", "Washington State",
    "West Virginia", "Western Kentucky", "Western Michigan", "Wisconsin"
]

In [13]:
team_name_map = [
    {"team_id": "001", "fbs_name": "Air Force", "pff_name": "AIR FORCE"},
    {"team_id": "002", "fbs_name": "Akron", "pff_name": "AKRON"},
    {"team_id": "003", "fbs_name": "Alabama", "pff_name": "ALABAMA"},
    {"team_id": "004", "fbs_name": "Appalachian State", "pff_name": "APP STATE"},
    {"team_id": "005", "fbs_name": "Arizona", "pff_name": "ARIZONA"},
    {"team_id": "006", "fbs_name": "Arizona State", "pff_name": "ARIZONA ST"},
    {"team_id": "007", "fbs_name": "Arkansas", "pff_name": "ARKANSAS"},
    {"team_id": "008", "fbs_name": "Arkansas State", "pff_name": "ARK STATE"},
    {"team_id": "009", "fbs_name": "Army", "pff_name": "ARMY"},
    {"team_id": "010", "fbs_name": "Auburn", "pff_name": "AUBURN"},
    {"team_id": "011", "fbs_name": "Ball State", "pff_name": "BALL ST"},
    {"team_id": "012", "fbs_name": "Baylor", "pff_name": "BAYLOR"},
    {"team_id": "013", "fbs_name": "Boise State", "pff_name": "BOISE ST"},
    {"team_id": "014", "fbs_name": "Boston College", "pff_name": "BOSTON COL"},
    {"team_id": "015", "fbs_name": "Bowling Green", "pff_name": "BOWL GREEN"},
    {"team_id": "016", "fbs_name": "Buffalo", "pff_name": "BUFFALO"},
    {"team_id": "017", "fbs_name": "BYU", "pff_name": "BYU"},
    {"team_id": "018", "fbs_name": "California", "pff_name": "CAL"},
    {"team_id": "019", "fbs_name": "Central Michigan", "pff_name": "C MICHIGAN"},
    {"team_id": "020", "fbs_name": "Charlotte", "pff_name": "CHARLOTTE"},
    {"team_id": "021", "fbs_name": "Cincinnati", "pff_name": "CINCINNATI"},
    {"team_id": "022", "fbs_name": "Clemson", "pff_name": "CLEMSON"},
    {"team_id": "023", "fbs_name": "Coastal Carolina", "pff_name": "COAST CAR"},
    {"team_id": "024", "fbs_name": "Colorado", "pff_name": "COLORADO"},
    {"team_id": "025", "fbs_name": "Colorado State", "pff_name": "COLO STATE"},
    {"team_id": "026", "fbs_name": "Connecticut", "pff_name": "UCONN"},
    {"team_id": "027", "fbs_name": "Duke", "pff_name": "DUKE"},
    {"team_id": "028", "fbs_name": "East Carolina", "pff_name": "E CAROLINA"},
    {"team_id": "029", "fbs_name": "Eastern Michigan", "pff_name": "E MICHIGAN"},
    {"team_id": "030", "fbs_name": "Florida", "pff_name": "FLORIDA"},
    {"team_id": "031", "fbs_name": "Florida Atlantic", "pff_name": "FAU"},
    {"team_id": "032", "fbs_name": "Florida International", "pff_name": "FIU"},
    {"team_id": "033", "fbs_name": "Florida State", "pff_name": "FLORIDA ST"},
    {"team_id": "034", "fbs_name": "Fresno State", "pff_name": "FRESNO ST"},
    {"team_id": "035", "fbs_name": "Georgia", "pff_name": "GEORGIA"},
    {"team_id": "036", "fbs_name": "Georgia Southern", "pff_name": "GA SOUTHRN"},
    {"team_id": "037", "fbs_name": "Georgia State", "pff_name": "GA STATE"},
    {"team_id": "038", "fbs_name": "Georgia Tech", "pff_name": "GA TECH"},
    {"team_id": "039", "fbs_name": "Hawai'i", "pff_name": "HAWAII"},
    {"team_id": "040", "fbs_name": "Houston", "pff_name": "HOUSTON"},
    {"team_id": "041", "fbs_name": "Illinois", "pff_name": "ILLINOIS"},
    {"team_id": "042", "fbs_name": "Indiana", "pff_name": "INDIANA"},
    {"team_id": "043", "fbs_name": "Iowa", "pff_name": "IOWA"},
    {"team_id": "044", "fbs_name": "Iowa State", "pff_name": "IOWA STATE"},
    {"team_id": "045", "fbs_name": "Jacksonville State", "pff_name": "JVILLE ST"},
    {"team_id": "046", "fbs_name": "James Madison", "pff_name": "JAMES MAD"},
    {"team_id": "047", "fbs_name": "Kansas", "pff_name": "KANSAS"},
    {"team_id": "048", "fbs_name": "Kansas State", "pff_name": "KANSAS ST"},
    {"team_id": "049", "fbs_name": "Kennesaw State", "pff_name": "KENNESAW"},
    {"team_id": "050", "fbs_name": "Kent State", "pff_name": "KENT STATE"},
    {"team_id": "051", "fbs_name": "Kentucky", "pff_name": "KENTUCKY"},
    {"team_id": "052", "fbs_name": "Liberty", "pff_name": "LIBERTY"},
    {"team_id": "053", "fbs_name": "Louisiana", "pff_name": "LA LAFAYET"},
    {"team_id": "054", "fbs_name": "Louisiana Monroe", "pff_name": "LA MONROE"},
    {"team_id": "055", "fbs_name": "Louisiana Tech", "pff_name": "LA TECH"},
    {"team_id": "056", "fbs_name": "Louisville", "pff_name": "LOUISVILLE"},
    {"team_id": "057", "fbs_name": "LSU", "pff_name": "LSU"},
    {"team_id": "058", "fbs_name": "Marshall", "pff_name": "MARSHALL"},
    {"team_id": "059", "fbs_name": "Maryland", "pff_name": "MARYLAND"},
    {"team_id": "060", "fbs_name": "Memphis", "pff_name": "MEMPHIS"},
    {"team_id": "061", "fbs_name": "Miami", "pff_name": "MIAMI FL"},
    {"team_id": "062", "fbs_name": "Miami (OH)", "pff_name": "MIAMI OH"},
    {"team_id": "063", "fbs_name": "Michigan", "pff_name": "MICHIGAN"},
    {"team_id": "064", "fbs_name": "Michigan State", "pff_name": "MICH STATE"},
    {"team_id": "065", "fbs_name": "Middle Tennessee", "pff_name": "MIDDLE TN"},
    {"team_id": "066", "fbs_name": "Minnesota", "pff_name": "MINNESOTA"},
    {"team_id": "067", "fbs_name": "Mississippi State", "pff_name": "MISS STATE"},
    {"team_id": "068", "fbs_name": "Missouri", "pff_name": "MISSOURI"},
    {"team_id": "069", "fbs_name": "Navy", "pff_name": "NAVY"},
    {"team_id": "070", "fbs_name": "NC State", "pff_name": "NC STATE"},
    {"team_id": "071", "fbs_name": "Nebraska", "pff_name": "NEBRASKA"},
    {"team_id": "072", "fbs_name": "Nevada", "pff_name": "NEVADA"},
    {"team_id": "073", "fbs_name": "New Mexico", "pff_name": "NEW MEXICO"},
    {"team_id": "074", "fbs_name": "New Mexico State", "pff_name": "NEW MEX ST"},
    {"team_id": "075", "fbs_name": "North Carolina", "pff_name": "N CAROLINA"},
    {"team_id": "076", "fbs_name": "North Texas", "pff_name": "N TEXAS"},
    {"team_id": "077", "fbs_name": "Northern Illinois", "pff_name": "N ILLINOIS"},
    {"team_id": "078", "fbs_name": "Northwestern", "pff_name": "NWESTERN"},
    {"team_id": "079", "fbs_name": "Notre Dame", "pff_name": "NOTRE DAME"},
    {"team_id": "080", "fbs_name": "Ohio", "pff_name": "OHIO"},
    {"team_id": "081", "fbs_name": "Ohio State", "pff_name": "OHIO STATE"},
    {"team_id": "082", "fbs_name": "Oklahoma", "pff_name": "OKLAHOMA"},
    {"team_id": "083", "fbs_name": "Oklahoma State", "pff_name": "OKLA STATE"},
    {"team_id": "084", "fbs_name": "Old Dominion", "pff_name": "DOMINION"},
    {"team_id": "085", "fbs_name": "Ole Miss", "pff_name": "OLE MISS"},
    {"team_id": "086", "fbs_name": "Oregon", "pff_name": "OREGON"},
    {"team_id": "087", "fbs_name": "Oregon State", "pff_name": "OREGON ST"},
    {"team_id": "088", "fbs_name": "Penn State", "pff_name": "PENN STATE"},
    {"team_id": "089", "fbs_name": "Pittsburgh", "pff_name": "PITTSBURGH"},
    {"team_id": "090", "fbs_name": "Purdue", "pff_name": "PURDUE"},
    {"team_id": "091", "fbs_name": "Rice", "pff_name": "RICE"},
    {"team_id": "092", "fbs_name": "Rutgers", "pff_name": "RUTGERS"},
    {"team_id": "093", "fbs_name": "Sam Houston State", "pff_name": "SM HOUSTON"},
    {"team_id": "094", "fbs_name": "San Diego State", "pff_name": "S DIEGO ST"},
    {"team_id": "095", "fbs_name": "San José State", "pff_name": "S JOSE ST"},
    {"team_id": "096", "fbs_name": "SMU", "pff_name": "SMU"},
    {"team_id": "097", "fbs_name": "South Alabama", "pff_name": "S ALABAMA"},
    {"team_id": "098", "fbs_name": "South Carolina", "pff_name": "S CAROLINA"},
    {"team_id": "099", "fbs_name": "South Florida", "pff_name": "USF"},
    {"team_id": "100", "fbs_name": "Southern Mississippi", "pff_name": "SO MISS"},
    {"team_id": "101", "fbs_name": "Stanford", "pff_name": "STANFORD"},
    {"team_id": "102", "fbs_name": "Syracuse", "pff_name": "SYRACUSE"},
    {"team_id": "103", "fbs_name": "TCU", "pff_name": "TCU"},
    {"team_id": "104", "fbs_name": "Temple", "pff_name": "TEMPLE"},
    {"team_id": "105", "fbs_name": "Tennessee", "pff_name": "TENNESSEE"},
    {"team_id": "106", "fbs_name": "Texas", "pff_name": "TEXAS"},
    {"team_id": "107", "fbs_name": "Texas A&M", "pff_name": "TEXAS A&M"},
    {"team_id": "108", "fbs_name": "Texas State", "pff_name": "TEXAS ST"},
    {"team_id": "109", "fbs_name": "Texas Tech", "pff_name": "TEXAS TECH"},
    {"team_id": "110", "fbs_name": "Toledo", "pff_name": "TOLEDO"},
    {"team_id": "111", "fbs_name": "Troy", "pff_name": "TROY"},
    {"team_id": "112", "fbs_name": "Tulane", "pff_name": "TULANE"},
    {"team_id": "113", "fbs_name": "Tulsa", "pff_name": "TULSA"},
    {"team_id": "114", "fbs_name": "UAB", "pff_name": "UAB"},
    {"team_id": "115", "fbs_name": "UCF", "pff_name": "UCF"},
    {"team_id": "116", "fbs_name": "UCLA", "pff_name": "UCLA"},
    {"team_id": "117", "fbs_name": "UMass", "pff_name": "UMASS"},
    {"team_id": "118", "fbs_name": "UNLV", "pff_name": "UNLV"},
    {"team_id": "119", "fbs_name": "USC", "pff_name": "USC"},
    {"team_id": "120", "fbs_name": "UTEP", "pff_name": "UTEP"},
    {"team_id": "121", "fbs_name": "UT San Antonio", "pff_name": "UTSA"},
    {"team_id": "122", "fbs_name": "Utah", "pff_name": "UTAH"},
    {"team_id": "123", "fbs_name": "Utah State", "pff_name": "UTAH ST"},
    {"team_id": "124", "fbs_name": "Vanderbilt", "pff_name": "VANDERBILT"},
    {"team_id": "125", "fbs_name": "Virginia", "pff_name": "VIRGINIA"},
    {"team_id": "126", "fbs_name": "Virginia Tech", "pff_name": "VA TECH"},
    {"team_id": "127", "fbs_name": "Wake Forest", "pff_name": "WAKE"},
    {"team_id": "128", "fbs_name": "Washington", "pff_name": "WASHINGTON"},
    {"team_id": "129", "fbs_name": "Washington State", "pff_name": "WASH STATE"},
    {"team_id": "130", "fbs_name": "West Virginia", "pff_name": "W VIRGINIA"},
    {"team_id": "131", "fbs_name": "Western Kentucky", "pff_name": "W KENTUCKY"},
    {"team_id": "132", "fbs_name": "Western Michigan", "pff_name": "W MICHIGAN"},
    {"team_id": "133", "fbs_name": "Wisconsin", "pff_name": "WISCONSIN"}
]

In [14]:
team_locations_map = {
    "Air Force": {"city": "Colorado Springs", "state": "CO"},
    "Akron": {"city": "Akron", "state": "OH"},
    "Alabama": {"city": "Tuscaloosa", "state": "AL"},
    "Appalachian State": {"city": "Boone", "state": "NC"},
    "Arizona": {"city": "Tucson", "state": "AZ"},
    "Arizona State": {"city": "Tempe", "state": "AZ"},
    "Arkansas": {"city": "Fayetteville", "state": "AR"},
    "Arkansas State": {"city": "Jonesboro", "state": "AR"},
    "Army": {"city": "West Point", "state": "NY"},
    "Auburn": {"city": "Auburn", "state": "AL"},
    "Ball State": {"city": "Muncie", "state": "IN"},
    "Baylor": {"city": "Waco", "state": "TX"},
    "Boise State": {"city": "Boise", "state": "ID"},
    "Boston College": {"city": "Chestnut Hill", "state": "MA"},
    "Bowling Green": {"city": "Bowling Green", "state": "OH"},
    "Buffalo": {"city": "Buffalo", "state": "NY"},
    "BYU": {"city": "Provo", "state": "UT"},
    "California": {"city": "Berkeley", "state": "CA"},
    "Central Michigan": {"city": "Mount Pleasant", "state": "MI"},
    "Charlotte": {"city": "Charlotte", "state": "NC"},
    "Cincinnati": {"city": "Cincinnati", "state": "OH"},
    "Clemson": {"city": "Clemson", "state": "SC"},
    "Coastal Carolina": {"city": "Conway", "state": "SC"},
    "Colorado": {"city": "Boulder", "state": "CO"},
    "Colorado State": {"city": "Fort Collins", "state": "CO"},
    "Connecticut": {"city": "Storrs", "state": "CT"},
    "Duke": {"city": "Durham", "state": "NC"},
    "East Carolina": {"city": "Greenville", "state": "NC"},
    "Eastern Michigan": {"city": "Ypsilanti", "state": "MI"},
    "Florida International": {"city": "Miami", "state": "FL"},
    "Florida": {"city": "Gainesville", "state": "FL"},
    "Florida Atlantic": {"city": "Boca Raton", "state": "FL"},
    "Florida State": {"city": "Tallahassee", "state": "FL"},
    "Fresno State": {"city": "Fresno", "state": "CA"},
    "Georgia": {"city": "Athens", "state": "GA"},
    "Georgia Southern": {"city": "Statesboro", "state": "GA"},
    "Georgia State": {"city": "Atlanta", "state": "GA"},
    "Georgia Tech": {"city": "Atlanta", "state": "GA"},
    "Hawai'i": {"city": "Honolulu", "state": "HI"},
    "Houston": {"city": "Houston", "state": "TX"},
    "Illinois": {"city": "Champaign", "state": "IL"},
    "Indiana": {"city": "Bloomington", "state": "IN"},
    "Iowa": {"city": "Iowa City", "state": "IA"},
    "Iowa State": {"city": "Ames", "state": "IA"},
    "Jacksonville State": {"city": "Jacksonville", "state": "AL"},
    "James Madison": {"city": "Harrisonburg", "state": "VA"},
    "Kansas": {"city": "Lawrence", "state": "KS"},
    "Kansas State": {"city": "Manhattan", "state": "KS"},
    "Kennesaw State": {"city": "Kennesaw", "state": "GA"},
    "Kent State": {"city": "Kent", "state": "OH"},
    "Kentucky": {"city": "Lexington", "state": "KY"},
    "Liberty": {"city": "Lynchburg", "state": "VA"},
    "Louisiana": {"city": "Lafayette", "state": "LA"},
    "Louisiana Monroe": {"city": "Monroe", "state": "LA"},
    "Louisiana Tech": {"city": "Ruston", "state": "LA"},
    "Louisville": {"city": "Louisville", "state": "KY"},
    "LSU": {"city": "Baton Rouge", "state": "LA"},
    "Marshall": {"city": "Huntington", "state": "WV"},
    "Maryland": {"city": "College Park", "state": "MD"},
    "Memphis": {"city": "Memphis", "state": "TN"},
    "Miami": {"city": "Coral Gables", "state": "FL"},
    "Miami (OH)": {"city": "Oxford", "state": "OH"},
    "Michigan": {"city": "Ann Arbor", "state": "MI"},
    "Michigan State": {"city": "East Lansing", "state": "MI"},
    "Middle Tennessee": {"city": "Murfreesboro", "state": "TN"},
    "Minnesota": {"city": "Minneapolis", "state": "MN"},
    "Mississippi State": {"city": "Starkville", "state": "MS"},
    "Missouri": {"city": "Columbia", "state": "MO"},
    "Navy": {"city": "Annapolis", "state": "MD"},
    "NC State": {"city": "Raleigh", "state": "NC"},
    "Nebraska": {"city": "Lincoln", "state": "NE"},
    "Nevada": {"city": "Reno", "state": "NV"},
    "New Mexico": {"city": "Albuquerque", "state": "NM"},
    "New Mexico State": {"city": "Las Cruces", "state": "NM"},
    "North Carolina": {"city": "Chapel Hill", "state": "NC"},
    "North Texas": {"city": "Denton", "state": "TX"},
    "Northern Illinois": {"city": "DeKalb", "state": "IL"},
    "Northwestern": {"city": "Evanston", "state": "IL"},
    "Notre Dame": {"city": "South Bend", "state": "IN"},
    "Ohio": {"city": "Athens", "state": "OH"},
    "Ohio State": {"city": "Columbus", "state": "OH"},
    "Oklahoma": {"city": "Norman", "state": "OK"},
    "Oklahoma State": {"city": "Stillwater", "state": "OK"},
    "Old Dominion": {"city": "Norfolk", "state": "VA"},
    "Ole Miss": {"city": "Oxford", "state": "MS"},
    "Oregon": {"city": "Eugene", "state": "OR"},
    "Oregon State": {"city": "Corvallis", "state": "OR"},
    "Penn State": {"city": "University Park", "state": "PA"},
    "Pittsburgh": {"city": "Pittsburgh", "state": "PA"},
    "Purdue": {"city": "West Lafayette", "state": "IN"},
    "Rice": {"city": "Houston", "state": "TX"},
    "Rutgers": {"city": "Piscataway", "state": "NJ"},
    "Sam Houston State": {"city": "Huntsville", "state": "TX"},
    "San Diego State": {"city": "San Diego", "state": "CA"},
    "San José State": {"city": "San Jose", "state": "CA"},
    "SMU": {"city": "Dallas", "state": "TX"},
    "South Alabama": {"city": "Mobile", "state": "AL"},
    "South Carolina": {"city": "Columbia", "state": "SC"},
    "South Florida": {"city": "Tampa", "state": "FL"},
    "Southern Mississippi": {"city": "Hattiesburg", "state": "MS"},
    "Stanford": {"city": "Stanford", "state": "CA"},
    "Syracuse": {"city": "Syracuse", "state": "NY"},
    "TCU": {"city": "Fort Worth", "state": "TX"},
    "Temple": {"city": "Philadelphia", "state": "PA"},
    "Tennessee": {"city": "Knoxville", "state": "TN"},
    "Texas": {"city": "Austin", "state": "TX"},
    "Texas A&M": {"city": "College Station", "state": "TX"},
    "Texas State": {"city": "San Marcos", "state": "TX"},
    "Texas Tech": {"city": "Lubbock", "state": "TX"},
    "Toledo": {"city": "Toledo", "state": "OH"},
    "Troy": {"city": "Troy", "state": "AL"},
    "Tulane": {"city": "New Orleans", "state": "LA"},
    "Tulsa": {"city": "Tulsa", "state": "OK"},
    "UAB": {"city": "Birmingham", "state": "AL"},
    "UCF": {"city": "Orlando", "state": "FL"},
    "UCLA": {"city": "Los Angeles", "state": "CA"},
    "UMass": {"city": "Amherst", "state": "MA"},
    "UNLV": {"city": "Las Vegas", "state": "NV"},
    "USC": {"city": "Los Angeles", "state": "CA"},
    "UTEP": {"city": "El Paso", "state": "TX"},
    "UT San Antonio": {"city": "San Antonio", "state": "TX"},
    "Utah": {"city": "Salt Lake City", "state": "UT"},
    "Utah State": {"city": "Logan", "state": "UT"},
    "Vanderbilt": {"city": "Nashville", "state": "TN"},
    "Virginia": {"city": "Charlottesville", "state": "VA"},
    "Virginia Tech": {"city": "Blacksburg", "state": "VA"},
    "Wake Forest": {"city": "Winston-Salem", "state": "NC"},
    "Washington": {"city": "Seattle", "state": "WA"},
    "Washington State": {"city": "Pullman", "state": "WA"},
    "West Virginia": {"city": "Morgantown", "state": "WV"},
    "Western Kentucky": {"city": "Bowling Green", "state": "KY"},
    "Western Michigan": {"city": "Kalamazoo", "state": "MI"},
    "Wisconsin": {"city": "Madison", "state": "WI"},
    "Wyoming": {"city": "Laramie", "state": "WY"}
}

In [15]:
team_conferences_map = {
    "Air Force": "Mountain West",
    "Akron": "MAC",
    "Alabama": "SEC",
    "Appalachian State": "Sun Belt",
    "Arizona": "Big 12",
    "Arizona State": "Big 12",
    "Arkansas": "SEC",
    "Arkansas State": "Sun Belt",
    "Army": "American",
    "Auburn": "SEC",
    "Ball State": "MAC",
    "Baylor": "Big 12",
    "Boise State": "Mountain West",
    "Boston College": "ACC",
    "Bowling Green": "MAC",
    "Buffalo": "MAC",
    "BYU": "Big 12",
    "California": "ACC",
    "Central Michigan": "MAC",
    "Charlotte": "American",
    "Cincinnati": "Big 12",
    "Clemson": "ACC",
    "Coastal Carolina": "Sun Belt",
    "Colorado": "Big 12",
    "Colorado State": "Mountain West",
    "Connecticut": "Independent",
    "Duke": "ACC",
    "East Carolina": "American",
    "Eastern Michigan": "MAC",
    "Florida International": "C-USA",
    "Florida": "SEC",
    "Florida Atlantic": "American",
    "Florida State": "ACC",
    "Fresno State": "Mountain West",
    "Georgia": "SEC",
    "Georgia Southern": "Sun Belt",
    "Georgia State": "Sun Belt",
    "Georgia Tech": "ACC",
    "Hawai'i": "Mountain West",
    "Houston": "Big 12",
    "Illinois": "Big Ten",
    "Indiana": "Big Ten",
    "Iowa": "Big Ten",
    "Iowa State": "Big 12",
    "Jacksonville State": "C-USA",
    "James Madison": "Sun Belt",
    "Kansas": "Big 12",
    "Kansas State": "Big 12",
    "Kennesaw State": "C-USA",
    "Kent State": "MAC",
    "Kentucky": "SEC",
    "Liberty": "C-USA",
    "Louisiana": "Sun Belt",
    "Louisiana Monroe": "Sun Belt",
    "Louisiana Tech": "C-USA",
    "Louisville": "ACC",
    "LSU": "SEC",
    "Marshall": "Sun Belt",
    "Maryland": "Big Ten",
    "Memphis": "American",
    "Miami": "ACC",
    "Miami (OH)": "MAC",
    "Michigan": "Big Ten",
    "Michigan State": "Big Ten",
    "Middle Tennessee": "C-USA",
    "Minnesota": "Big Ten",
    "Mississippi State": "SEC",
    "Missouri": "SEC",
    "Navy": "American",
    "NC State": "ACC",
    "Nebraska": "Big Ten",
    "Nevada": "Mountain West",
    "New Mexico": "Mountain West",
    "New Mexico State": "C-USA",
    "North Carolina": "ACC",
    "North Texas": "American",
    "Northern Illinois": "MAC",
    "Northwestern": "Big Ten",
    "Notre Dame": "Independent",
    "Ohio": "MAC",
    "Ohio State": "Big Ten",
    "Oklahoma": "SEC",
    "Oklahoma State": "Big 12",
    "Old Dominion": "Sun Belt",
    "Ole Miss": "SEC",
    "Oregon": "Big Ten",
    "Oregon State": "Independent",
    "Penn State": "Big Ten",
    "Pittsburgh": "ACC",
    "Purdue": "Big Ten",
    "Rice": "American",
    "Rutgers": "Big Ten",
    "Sam Houston State": "C-USA",
    "San Diego State": "Mountain West",
    "San José State": "Mountain West",
    "SMU": "ACC",
    "South Alabama": "Sun Belt",
    "South Carolina": "SEC",
    "South Florida": "American",
    "Southern Mississippi": "Sun Belt",
    "Stanford": "ACC",
    "Syracuse": "ACC",
    "TCU": "Big 12",
    "Temple": "American",
    "Tennessee": "SEC",
    "Texas": "SEC",
    "Texas A&M": "SEC",
    "Texas State": "Sun Belt",
    "Texas Tech": "Big 12",
    "Toledo": "MAC",
    "Troy": "Sun Belt",
    "Tulane": "American",
    "Tulsa": "American",
    "UAB": "American",
    "UCF": "Big 12",
    "UCLA": "Big Ten",
    "UMass": "Independent",
    "UNLV": "Mountain West",
    "USC": "Big Ten",
    "UTEP": "C-USA",
    "UT San Antonio": "American",
    "Utah": "Big 12",
    "Utah State": "Mountain West",
    "Vanderbilt": "SEC",
    "Virginia": "ACC",
    "Virginia Tech": "ACC",
    "Wake Forest": "ACC",
    "Washington": "Big Ten",
    "Washington State": "Independent",
    "West Virginia": "Big 12",
    "Western Kentucky": "C-USA",
    "Western Michigan": "MAC",
    "Wisconsin": "Big Ten",
    "Wyoming": "Mountain West"
}

# Create Dataframes for Insertion into Tables

In [149]:
pbp_df = pd.read_csv("2024_pbp.csv")
pff_passing = pd.read_csv("2024_pff_passing.csv")
pff_rushing = pd.read_csv("2024_pff_rushing.csv")
pff_receiving = pd.read_csv("2024_pff_receiving.csv")
pff_blocking = pd.read_csv("2024_pff_blocking.csv")
pff_defense = pd.read_csv("2024_pff_defense.csv")

C:\Users\lehma\AppData\Local\Temp\ipykernel_30168\3178465243.py:1: DtypeWarning: Columns (73,132,133,137,145,146,149,150,161,224,240,241,242,243,244,245,246,247,248,249,250,252,253,255,259,262,265,268,271,274,277,280,283,286,289,292,295) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_df = pd.read_csv("2024_pbp.csv")


In [150]:
# renumber unique drive IDs
unique_drives = pbp_df['drive_id'].unique()
drive_id_map = {old_id: new_id for new_id, old_id in enumerate(unique_drives)}

# Map the original drive_ids to new drive_ids
pbp_df['drive_id'] = pbp_df['drive_id'].map(drive_id_map)

#Same for game IDs
game_id_map = {old_id: new_id for new_id, old_id in enumerate(pbp_df['game_id'].unique())}
pbp_df['game_id'] = pbp_df['game_id'].map(game_id_map)

In [151]:
# teams

# Get all unique team names from home and away columns
all_teams = pd.Series(pd.concat([pbp_df["home"], pbp_df["away"]])).dropna().unique()

# Sort alphabetically and assign team_id starting from 0
sorted_teams = sorted(all_teams)
team_id_lookup = {team: i for i, team in enumerate(sorted_teams)}

# Build teams data
teams_data = []
for team in sorted_teams:
    team_id = team_id_lookup[team]
    location = team_locations_map.get(team, {"city": None, "state": None})
    conference = team_conferences_map.get(team, "non-FBS")

    teams_data.append({
        "team_id": team_id,
        "team_name": team,
        "conference": conference,
        "city": location["city"],
        "state": location["state"]
    })

# Create teams_df
teams_df = pd.DataFrame(teams_data)

# Cast columns to appropriate types
teams_df = teams_df.astype({
    "team_id": "int",
    "team_name": "string",
    "conference": "string",
    "city": "string",
    "state": "string"
})

# Step 6: Reorder columns
teams_df = teams_df[["team_id", "team_name", "conference", "city", "state"]]

In [152]:
print(teams_df)

     team_id          team_name     conference              city state
0          0  Abilene Christian        non-FBS              <NA>  <NA>
1          1          Air Force  Mountain West  Colorado Springs    CO
2          2              Akron            MAC             Akron    OH
3          3            Alabama            SEC        Tuscaloosa    AL
4          4        Alabama A&M        non-FBS              <NA>  <NA>
..       ...                ...            ...               ...   ...
294      294          Wisconsin        Big Ten           Madison    WI
295      295            Wofford        non-FBS              <NA>  <NA>
296      296            Wyoming  Mountain West           Laramie    WY
297      297               Yale        non-FBS              <NA>  <NA>
298      298   Youngstown State        non-FBS              <NA>  <NA>

[299 rows x 5 columns]


In [153]:
# updating the team_name_map according to teams_df

# Step 1: Build a lookup from team_name in teams_df to team_id
teams_id_lookup = dict(zip(teams_df["team_name"], teams_df["team_id"]))

# Step 2: Update team_id in team_name_map (only if team is in teams_df)
for entry in team_name_map:
    fbs_name = entry["fbs_name"]
    if fbs_name in teams_id_lookup:
        entry["team_id"] = str(teams_id_lookup[fbs_name])

In [154]:
# games

# Sort plays so final plays are at the end
pbp_sorted = pbp_df.sort_values(["game_id", "game_play_number"])

# Get final play of each game
final_plays = pbp_sorted.groupby("game_id").tail(1)

# Extract basic game info from first play
game_info = pbp_sorted.groupby("game_id").head(1)[
    ["game_id", "year", "week", "home", "away"]
]

# Determine home/away scores from final play
def determine_scores(row):
    if row["pos_team"] == row["home"]:
        home_score = row["pos_team_score"]
        away_score = row["def_pos_team_score"]
    else:
        home_score = row["def_pos_team_score"]
        away_score = row["pos_team_score"]
    return pd.Series({"home_score": home_score, "away_score": away_score})

# Apply logic row-wise
final_scores = final_plays.apply(determine_scores, axis=1)
final_scores["game_id"] = final_plays["game_id"].values  # keep game_id for merging

# Merge scores into game_info
games_df = pd.merge(game_info, final_scores, on="game_id")

# Map home and away teams to IDs using teams_df
team_id_lookup = dict(zip(teams_df["team_name"], teams_df["team_id"]))
games_df["home_team_id"] = games_df["home"].map(team_id_lookup)
games_df["away_team_id"] = games_df["away"].map(team_id_lookup)

# Ensure column order matches the schema
games_df = games_df[
    ["game_id", "year", "week", "home_team_id", "home", "away_team_id", "away", "home_score", "away_score"]
]

# Cast column types to match schema
games_df = games_df.astype({
    "game_id": "int64",          
    "year": "int",              
    "week": "int",              
    "home_team_id": "int",     
    "home": "string",         
    "away_team_id": "int",     
    "away": "string",         
    "home_score": "Int64",  
    "away_score": "Int64"     
})

# Reset index
games_df = games_df.reset_index(drop=True)

In [155]:
print(games_df)

      game_id  year  week  home_team_id                home  away_team_id  \
0           0  2024     1             3             Alabama           289   
1           1  2024     1            10            Arkansas            12   
2           2  2024     1            14              Auburn             4   
3           3  2024     1            69             Florida           137   
4           4  2024     1            80             Georgia            45   
...       ...   ...   ...           ...                 ...           ...   
1585     1585  2024    15           240               Texas            80   
1586     1586  2024    15           123           Louisiana           128   
1587     1587  2024    15           103       Jackson State           221   
1588     1588  2024    15           168  North Dakota State             0   
1589     1589  2024    15           217        South Dakota           235   

                     away  home_score  away_score  
0        Western Kentuc

In [156]:
#drives

# Define columns you want (make sure names match your dataframe)
drive_cols = [
    "drive_id", "game_id", "drive_number", "pos_team",
    "drive_result", "drive_yards",
    "drive_start_yards_to_goal", "drive_end_yards_to_goal",
    "drive_pts",
    "drive_time_minutes_start", "drive_time_seconds_start",
    "drive_time_minutes_end", "drive_time_seconds_end",
    "drive_time_minutes_elapsed", "drive_time_seconds_elapsed"
]

# Take first row per drive
drives_df = pbp_df.groupby("drive_id").first().reset_index()

# Select only the columns you want
drives_df = drives_df[drive_cols]

# Map pos_team to pos_team_id
team_id_lookup = {entry["fbs_name"]: int(entry["team_id"]) for entry in team_name_map}
drives_df["pos_team_id"] = drives_df["pos_team"].map(team_id_lookup)

# Filter drives where pos_team is in fbs_teams
drives_df = drives_df[drives_df["pos_team"].isin(fbs_teams)]

# Filter to only include valid game_ids
drives_df = drives_df[drives_df["game_id"].isin(games_df["game_id"])]

# Enforce proper dtypes
drives_df = drives_df.astype({
    "drive_id": "int64",              
    "game_id": "int64",           
    "drive_number": "Int64",         
    "pos_team_id": "Int64",      
    "pos_team": "string",      
    "drive_result": "string",   
    "drive_yards": "Int64",     
    "drive_start_yards_to_goal": "Int64", 
    "drive_end_yards_to_goal": "Int64",    
    "drive_pts": "Int64",        
    "drive_time_minutes_start": "Int64",   
    "drive_time_seconds_start": "Int64",   
    "drive_time_minutes_end": "Int64",    
    "drive_time_seconds_end": "Int64",   
    "drive_time_minutes_elapsed": "Int64", 
    "drive_time_seconds_elapsed": "Int64" 
})

# Rename columns to match schema
drives_df = drives_df.rename(columns={
    "drive_number": "drive_num",
    "drive_pts": "drive_points"
})

# Reorder columns to match insert order
drives_df = drives_df[
    ["drive_id", "game_id", "drive_num", "pos_team_id", "pos_team", "drive_result",
     "drive_yards", "drive_start_yards_to_goal", "drive_end_yards_to_goal", "drive_points",
     "drive_time_minutes_start", "drive_time_seconds_start", "drive_time_minutes_end",
     "drive_time_seconds_end", "drive_time_minutes_elapsed", "drive_time_seconds_elapsed"]
]

In [157]:
print(drives_df)

       drive_id  game_id  drive_num  pos_team_id          pos_team  \
0             0        0          1          289  Western Kentucky   
1             1        0          2            3           Alabama   
2             2        0          3          289  Western Kentucky   
3             3        0          4            3           Alabama   
4             4        0          5          289  Western Kentucky   
...         ...      ...        ...          ...               ...   
37272     37272     1586         18          128          Marshall   
37273     37273     1586         19          123         Louisiana   
37274     37274     1586         20          128          Marshall   
37275     37275     1586         21          123         Louisiana   
37276     37276     1586         22          128          Marshall   

      drive_result  drive_yards  drive_start_yards_to_goal  \
0             PUNT            6                         75   
1             PUNT           -3    

In [158]:
# Plays

# Get team_id map from teams_df (new numbering)
team_id_lookup = dict(zip(teams_df["team_name"], teams_df["team_id"]))

# Select play columns
play_cols = [
    "id_play", "game_id", "game_play_number", "drive_id", "half", "period",
    "clock_minutes", "clock_seconds", "TimeSecsRem",
    "pos_team", "def_pos_team",
    "play_type", "play_text", "pass", "pass_attempt", "completion", "rush",
    "yards_gained", "down", "distance", "yards_to_goal",
    "EPA", "def_EPA", "wpa", "ep_before", "ep_after", "wp_before", "wp_after",
    "sack", "int", "penalty_detail", "yds_penalty", "firstD_by_penalty",
    "firstD_by_yards", "success", "turnover", "downs_turnover", "score_pts",
    "scoring_play", "defense_score_play", "change_of_poss", "off_timeout_called",
    "off_timeouts_rem_before", "def_timeout_called", "def_timeouts_rem_before"
]

# Subset from full PBP
plays_df = pbp_df[play_cols].copy()

# Rename for schema compatibility
plays_df = plays_df.rename(columns={
    "TimeSecsRem": "time_secs_rem",
    "EPA": "epa",
    "def_EPA": "def_epa",
    "firstD_by_penalty": "firstd_by_penalty",
    "firstD_by_yards": "firstd_by_yards",
    "downs_turnover": "turnover_downs",
    "off_timeout_called": "off_timeout",
    "def_timeout_called": "def_timeout"
})

# Map team names to team IDs from teams_df
plays_df["pos_team_id"] = plays_df["pos_team"].map(team_id_lookup)
plays_df["def_pos_team_id"] = plays_df["def_pos_team"].map(team_id_lookup)

# Filter: keep only plays from valid drives and games
plays_df = plays_df[
    (plays_df["drive_id"].isin(drives_df["drive_id"])) &
    (plays_df["game_id"].isin(games_df["game_id"])) &
    (plays_df["pos_team_id"].notnull() | plays_df["def_pos_team_id"].notnull())
].copy()

plays_df["id_play"] = range(len(plays_df))

# Set dtypes for SQL schema
plays_df = plays_df.astype({
    "id_play": "int",
    "game_id": "int",
    "game_play_number": "int",
    "drive_id": "int",
    "half": "int",
    "period": "int",
    "clock_minutes": "int",
    "clock_seconds": "int",
    "time_secs_rem": "int",
    "pos_team_id": "Int64",
    "def_pos_team_id": "Int64",
    "pass": "Int64",
    "pass_attempt": "Int64",
    "completion": "Int64",
    "rush": "Int64",
    "yards_gained": "Int64",
    "down": "Int64",
    "distance": "Int64",
    "yards_to_goal": "Int64",
    "epa": "float",
    "def_epa": "float",
    "wpa": "float",
    "ep_before": "float",
    "ep_after": "float",
    "wp_before": "float",
    "wp_after": "float",
    "sack": "Int64",
    "int": "Int64",
    "yds_penalty": "Int64",
    "firstd_by_penalty": "Int64",
    "firstd_by_yards": "Int64",
    "success": "Int64",
    "turnover": "Int64",
    "turnover_downs": "Int64",
    "score_pts": "Int64",
    "scoring_play": "Int64",
    "defense_score_play": "Int64",
    "change_of_poss": "Int64",
    "off_timeout": "Int64",
    "off_timeouts_rem_before": "Int64",
    "def_timeout": "Int64",
    "def_timeouts_rem_before": "Int64"
})

# Final column ordering to match schema
plays_df = plays_df[
    [
        "id_play", "game_id", "game_play_number", "drive_id", "half", "period",
        "clock_minutes", "clock_seconds", "time_secs_rem",
        "pos_team", "pos_team_id", "def_pos_team", "def_pos_team_id",
        "play_type", "play_text", "pass", "pass_attempt", "completion", "rush",
        "yards_gained", "down", "distance", "yards_to_goal",
        "epa", "def_epa", "wpa", "ep_before", "ep_after", "wp_before", "wp_after",
        "sack", "int", "penalty_detail", "yds_penalty", "firstd_by_penalty",
        "firstd_by_yards", "success", "turnover", "turnover_downs", "score_pts",
        "scoring_play", "defense_score_play", "change_of_poss", "off_timeout",
        "off_timeouts_rem_before", "def_timeout", "def_timeouts_rem_before"
    ]
]

In [159]:
print(plays_df)

        id_play  game_id  game_play_number  drive_id  half  period  \
0             0        0                 1         0     1       1   
1             1        0                 2         0     1       1   
2             2        0                 3         0     1       1   
3             3        0                 4         0     1       1   
4             4        0                 5         0     1       1   
...         ...      ...               ...       ...   ...     ...   
546798   282021     1586               148     37275     2       4   
546799   282022     1586               149     37275     2       4   
546800   282023     1586               150     37276     2       4   
546801   282024     1586               151     37276     2       4   
546802   282025     1586               151     37276     2       4   

        clock_minutes  clock_seconds  time_secs_rem          pos_team  ...  \
0                  15              0           1800  Western Kentucky  ...   
1  

In [175]:
# Passing

# Step 1: Build lookup from pff_name → team_id
team_id_lookup = {entry["pff_name"]: int(entry["team_id"]) for entry in team_name_map}

# Step 2: Select and copy relevant columns
pass_cols = [
    "team_name", "attempts", "avg_depth_of_target", "avg_time_to_throw",
    "big_time_throws", "completion_percent", "completions", "def_gen_pressures",
    "dropbacks", "drops", "grades_offense", "grades_pass", "grades_run",
    "hit_as_threw", "interceptions", "sacks", "scrambles", "thrown_aways",
    "touchdowns", "turnover_worthy_plays", "yards"
]
pff_passing_df = pff_passing[pass_cols].copy()

# Step 3: Rename columns to match schema
pff_passing_df.rename(columns={
    "completion_percent": "completion_percentage",
    "grades_offense": "grade_offense",
    "grades_pass": "grade_passing",
    "grades_run": "grade_run",
    "thrown_aways": "thrown_away"
}, inplace=True)

# Step 4: Add team_id from team_name map
pff_passing_df["team_id"] = pff_passing_df["team_name"].map(team_id_lookup)

# Step 5: Drop teams with no matched ID
pff_passing_df = pff_passing_df.dropna(subset=["team_id"]).copy()

# Step 6: Compute weighted-mode depth of target (from max attempts row per team)
def weighted_mode(df):
    idx = df["attempts"].idxmax()
    return df.loc[idx, "avg_depth_of_target"]

avg_depth_map = pff_passing_df.groupby("team_name").apply(weighted_mode).rename("avg_depth_of_target")

# Step 7: Aggregate by team_name
aggregated = pff_passing_df.groupby("team_name").agg({
    "attempts": "sum",
    "avg_time_to_throw": "mean",
    "big_time_throws": "sum",
    "completions": "sum",
    "def_gen_pressures": "sum",
    "dropbacks": "sum",
    "drops": "sum",
    "grade_offense": "mean",
    "grade_passing": "mean",
    "grade_run": "mean",
    "hit_as_threw": "sum",
    "interceptions": "sum",
    "sacks": "sum",
    "scrambles": "sum",
    "thrown_away": "sum",
    "touchdowns": "sum",
    "turnover_worthy_plays": "sum",
    "yards": "sum"
}).reset_index()

# Step 8: Merge avg_depth back in
aggregated = aggregated.merge(avg_depth_map, on="team_name")

# Step 9: Compute derived stats
aggregated["completion_percentage"] = aggregated["completions"] / aggregated["attempts"]

# Step 10: Add team_id again (from team_name to ensure match)
aggregated["team_id"] = aggregated["team_name"].map(team_id_lookup)

# Step 11: Reorder and finalize DataFrame
pff_passing_df = aggregated[[
    "team_id", "team_name", "attempts", "avg_depth_of_target", "avg_time_to_throw",
    "big_time_throws", "completion_percentage", "completions", "def_gen_pressures",
    "dropbacks", "drops", "grade_offense", "grade_passing", "grade_run",
    "hit_as_threw", "interceptions", "sacks", "scrambles", "thrown_away",
    "touchdowns", "turnover_worthy_plays", "yards"
]]

# Step 12: Final type casting
pff_passing_df = pff_passing_df.astype({
    "team_id": "int",
    "team_name": "string",
    "attempts": "Int64",
    "avg_depth_of_target": "float",
    "avg_time_to_throw": "float",
    "big_time_throws": "Int64",
    "completion_percentage": "float",
    "completions": "Int64",
    "def_gen_pressures": "Int64",
    "dropbacks": "Int64",
    "drops": "Int64",
    "grade_offense": "float",
    "grade_passing": "float",
    "grade_run": "float",
    "hit_as_threw": "Int64",
    "interceptions": "Int64",
    "sacks": "Int64",
    "scrambles": "Int64",
    "thrown_away": "Int64",
    "touchdowns": "Int64",
    "turnover_worthy_plays": "Int64",
    "yards": "Int64"
})

In [176]:
# Identify all rows with duplicate team_id
duplicate_rows = pff_passing_df[pff_passing_df.duplicated(subset="team_id", keep=False)]

# Print them
print(duplicate_rows)

    team_id  team_name  attempts  avg_depth_of_target  avg_time_to_throw  \
40       95   ILLINOIS       355                 10.5             2.5450   
94       95  S JOSE ST       555                 12.4             2.7425   

    big_time_throws  completion_percentage  completions  def_gen_pressures  \
40               20               0.600000          213                155   
94               31               0.569369          316                143   

    dropbacks  ...  grade_passing  grade_run  hit_as_threw  interceptions  \
40        416  ...           72.6       63.6             4              6   
94        584  ...           68.4       59.6             4             16   

    sacks  scrambles  thrown_away  touchdowns  turnover_worthy_plays  yards  
40     30         31           21          22                     14   2754  
94     11         18           21          31                     22   4177  

[2 rows x 22 columns]


In [161]:
#pff_rushing

# Create a lookup dictionary from team_name_map: pff_name to team_id
team_id_lookup = {entry["pff_name"]: int(entry["team_id"]) for entry in team_name_map}

# Select and rename relevant columns
rush_cols = [
    "team_name", "attempts", "avoided_tackles", "breakaway_yards", "first_downs",
    "fumbles", "grades_offense", "grades_pass_block", "grades_run", "rec_yards",
    "receptions", "routes", "scramble_yards", "targets", "touchdowns", "yards",
    "yards_after_contact"
]

pff_rushing_df = pff_rushing[rush_cols].copy()

# Drop teams not in the team_id lookup
pff_rushing_df["team_id"] = pff_rushing_df["team_name"].map(team_id_lookup)
pff_rushing_df = pff_rushing_df.dropna(subset=["team_id"]).copy()

# Group and aggregate as specified
pff_rushing_agg = pff_rushing_df.groupby("team_name").agg({
    "attempts": "sum",
    "avoided_tackles": "sum",
    "breakaway_yards": "sum",
    "first_downs": "sum",
    "fumbles": "sum",
    "grades_offense": "mean",
    "grades_pass_block": "mean",
    "grades_run": "mean",
    "rec_yards": "sum",
    "receptions": "sum",
    "routes": "sum",
    "scramble_yards": "sum",
    "targets": "sum",
    "touchdowns": "sum",
    "yards": "sum",
    "yards_after_contact": "sum"
}).reset_index()

# Map team_id again after grouping
pff_rushing_agg["team_id"] = pff_rushing_agg["team_name"].map(team_id_lookup)

# Reorder columns
pff_rushing_df = pff_rushing_agg[
    ["team_id", "team_name", "attempts", "avoided_tackles", "breakaway_yards",
     "first_downs", "fumbles", "grades_offense", "grades_pass_block", "grades_run",
     "rec_yards", "receptions", "routes", "scramble_yards", "targets", "touchdowns",
     "yards", "yards_after_contact"]
]

# Ensure correct data types
pff_rushing_df = pff_rushing_df.astype({
    "team_id": "int",
    "team_name": "string",
    "attempts": "Int64",
    "avoided_tackles": "Int64",
    "breakaway_yards": "Int64",
    "first_downs": "Int64",
    "fumbles": "Int64",
    "grades_offense": "float",
    "grades_pass_block": "float",
    "grades_run": "float",
    "rec_yards": "Int64",
    "receptions": "Int64",
    "routes": "Int64",
    "scramble_yards": "Int64",
    "targets": "Int64",
    "touchdowns": "Int64",
    "yards": "Int64",
    "yards_after_contact": "Int64"
})

In [162]:
# pff_receiving

# Create a lookup dictionary from team_name_map: pff_name → team_id
team_id_lookup = {entry["pff_name"]: int(entry["team_id"]) for entry in team_name_map}

# Define relevant columns
recv_cols = [
    "team_name", "contested_receptions", "contested_targets", "grades_offense",
    "grades_pass_route", "receptions", "routes", "yards", "yards_after_catch"
]

# Subset the data
pff_receiving_df = pff_receiving[recv_cols].copy()

# Drop teams not in the mapping
pff_receiving_df["team_id"] = pff_receiving_df["team_name"].map(team_id_lookup)
pff_receiving_df = pff_receiving_df.dropna(subset=["team_id"]).copy()

# Group and aggregate by team_name
pff_receiving_agg = pff_receiving_df.groupby("team_name").agg({
    "contested_receptions": "sum",
    "contested_targets": "sum",
    "grades_offense": "mean",
    "grades_pass_route": "mean",
    "receptions": "sum",
    "routes": "sum",
    "yards": "sum",
    "yards_after_catch": "sum"
}).reset_index()

# Map team_id again post-grouping
pff_receiving_agg["team_id"] = pff_receiving_agg["team_name"].map(team_id_lookup)

# Reorder columns
pff_receiving_df = pff_receiving_agg[
    ["team_id", "team_name", "contested_receptions", "contested_targets",
     "grades_offense", "grades_pass_route", "receptions", "routes", "yards",
     "yards_after_catch"]
]

# Ensure correct data types
pff_receiving_df = pff_receiving_df.astype({
    "team_id": "int",
    "team_name": "string",
    "contested_receptions": "Int64",
    "contested_targets": "Int64",
    "grades_offense": "float",
    "grades_pass_route": "float",
    "receptions": "Int64",
    "routes": "Int64",
    "yards": "Int64",
    "yards_after_catch": "Int64"
})

In [163]:
# pff_blocking

team_id_lookup = {entry["pff_name"]: int(entry["team_id"]) for entry in team_name_map}

block_cols = [
    "team_name", "grades_offense", "grades_pass_block", "grades_run_block",
    "hits_allowed", "hurries_allowed", "penalties",
    "pressures_allowed", "sacks_allowed"
]

pff_blocking_df = pff_blocking[block_cols].copy()
pff_blocking_df["team_id"] = pff_blocking_df["team_name"].map(team_id_lookup)

pff_blocking_df = pff_blocking_df.dropna(subset=["team_id"])

pff_blocking_df = pff_blocking_df.groupby("team_name", as_index=False).agg({
    "grades_offense": "mean",
    "grades_pass_block": "mean",
    "grades_run_block": "mean",
    "hits_allowed": "sum",
    "hurries_allowed": "sum",
    "penalties": "sum",
    "pressures_allowed": "sum",
    "sacks_allowed": "sum",
    "team_id": "first"
})

pff_blocking_df = pff_blocking_df.astype({
    "team_id": "int",
    "team_name": "string",
    "grades_offense": "float",
    "grades_pass_block": "float",
    "grades_run_block": "float",
    "hits_allowed": "Int64",
    "hurries_allowed": "Int64",
    "penalties": "Int64",
    "pressures_allowed": "Int64",
    "sacks_allowed": "Int64"
})

pff_blocking_df = pff_blocking_df[
    ["team_id", "team_name", "grades_offense", "grades_pass_block", "grades_run_block",
     "hits_allowed", "hurries_allowed", "penalties", "pressures_allowed", "sacks_allowed"]
]

In [164]:
# pff_defense

team_id_lookup = {entry["pff_name"]: int(entry["team_id"]) for entry in team_name_map}

def_cols = [
    "team_name", "batted_passes", "forced_fumbles", "fumble_recoveries",
    "grades_coverage_defense", "grades_defense", "grades_pass_rush_defense",
    "grades_run_defense", "grades_tackle", "hits", "hurries", "interceptions",
    "missed_tackles", "pass_break_ups", "penalties", "qb_rating_against",
    "receptions", "sacks", "safeties", "stops", "tackles", "tackles_for_loss",
    "total_pressures", "touchdowns", "yards", "yards_after_catch"
]

pff_defense_df = pff_defense[def_cols].copy()
pff_defense_df["team_id"] = pff_defense_df["team_name"].map(team_id_lookup)

pff_defense_df = pff_defense_df.dropna(subset=["team_id"])

pff_defense_df = pff_defense_df.groupby("team_name", as_index=False).agg({
    "batted_passes": "sum",
    "forced_fumbles": "sum",
    "fumble_recoveries": "sum",
    "grades_coverage_defense": "mean",
    "grades_defense": "mean",
    "grades_pass_rush_defense": "mean",
    "grades_run_defense": "mean",
    "grades_tackle": "mean",
    "hits": "sum",
    "hurries": "sum",
    "interceptions": "sum",
    "missed_tackles": "sum",
    "pass_break_ups": "sum",
    "penalties": "sum",
    "qb_rating_against": "mean",
    "receptions": "sum",
    "sacks": "sum",
    "safeties": "sum",
    "stops": "sum",
    "tackles": "sum",
    "tackles_for_loss": "sum",
    "total_pressures": "sum",
    "touchdowns": "sum",
    "yards": "sum",
    "yards_after_catch": "sum",
    "team_id": "first"
})

pff_defense_df = pff_defense_df.astype({
    "team_id": "int",
    "team_name": "string",
    "batted_passes": "Int64",
    "forced_fumbles": "Int64",
    "fumble_recoveries": "Int64",
    "grades_coverage_defense": "float",
    "grades_defense": "float",
    "grades_pass_rush_defense": "float",
    "grades_run_defense": "float",
    "grades_tackle": "float",
    "hits": "Int64",
    "hurries": "Int64",
    "interceptions": "Int64",
    "missed_tackles": "Int64",
    "pass_break_ups": "Int64",
    "penalties": "Int64",
    "qb_rating_against": "float",
    "receptions": "Int64",
    "sacks": "float",
    "safeties": "Int64",
    "stops": "Int64",
    "tackles": "Int64",
    "tackles_for_loss": "float",
    "total_pressures": "Int64",
    "touchdowns": "Int64",
    "yards": "Int64",
    "yards_after_catch": "Int64"
})

pff_defense_df = pff_defense_df[
    ["team_id", "team_name", "batted_passes", "forced_fumbles", "fumble_recoveries",
     "grades_coverage_defense", "grades_defense", "grades_pass_rush_defense",
     "grades_run_defense", "grades_tackle", "hits", "hurries", "interceptions",
     "missed_tackles", "pass_break_ups", "penalties", "qb_rating_against",
     "receptions", "sacks", "safeties", "stops", "tackles", "tackles_for_loss",
     "total_pressures", "touchdowns", "yards", "yards_after_catch"]
]

In [165]:
# EPA

# Define situational filters
redzone_filter = plays_df["yards_to_goal"] <= 20
past40_filter = plays_df["yards_to_goal"] <= 40
penalty_filter = plays_df["penalty_detail"].notna()
timeout_filter = (plays_df["off_timeout"] == 1) | (plays_df["def_timeout"] == 1)

first_down = plays_df["down"] == 1
second_down = plays_df["down"] == 2
third_down = plays_df["down"] == 3
fourth_down = plays_df["down"] == 4

# Helper function to calculate total and per-play EPA 
def calc_epa_stats(df, group_col, epa_col):
    total = df.groupby(group_col)[epa_col].sum()
    per_play = df.groupby(group_col)[epa_col].mean()
    return total, per_play

# Build epa_stats DataFrame 
epa_stats = pd.DataFrame()

# Basic EPA
epa_stats["total_epa"], epa_stats["epa_per_play"] = calc_epa_stats(plays_df, "pos_team_id", "epa")
epa_stats["def_total_epa"], epa_stats["def_epa_per_play"] = calc_epa_stats(plays_df, "def_pos_team_id", "def_epa")

# Passing EPA
pass_plays = plays_df[plays_df["pass"] == 1]
epa_stats["passing_total_epa"], epa_stats["passing_epa_per_play"] = calc_epa_stats(pass_plays, "pos_team_id", "epa")
epa_stats["def_passing_epa_total"], epa_stats["def_passing_epa_per_play"] = calc_epa_stats(pass_plays, "def_pos_team_id", "def_epa")

# Rushing EPA
rush_plays = plays_df[plays_df["rush"] == 1]
epa_stats["rushing_total_epa"], epa_stats["rushing_epa_per_play"] = calc_epa_stats(rush_plays, "pos_team_id", "epa")
epa_stats["def_rushing_total_epa"], epa_stats["def_rushing_epa_per_play"] = calc_epa_stats(rush_plays, "def_pos_team_id", "def_epa")

# Down-specific EPA
for down_filter, label in zip([first_down, second_down, third_down, fourth_down],
                              ["first", "second", "third", "fourth"]):
    down_df = plays_df[down_filter]
    epa_stats[f"{label}down_epa_per_play"] = down_df.groupby("pos_team_id")["epa"].mean()
    epa_stats[f"def_{label}down_epa_per_play"] = down_df.groupby("def_pos_team_id")["def_epa"].mean()

# Red zone EPA
rz_df = plays_df[redzone_filter]
epa_stats["redzone_total_epa"], epa_stats["redzone_epa_per_play"] = calc_epa_stats(rz_df, "pos_team_id", "epa")
epa_stats["def_redzone_total_epa"], epa_stats["def_redzone_epa_per_play"] = calc_epa_stats(rz_df, "def_pos_team_id", "def_epa")

# Past 40-yard line EPA
p40_df = plays_df[past40_filter]
epa_stats["past40_total_epa"], epa_stats["past40_epa_per_play"] = calc_epa_stats(p40_df, "pos_team_id", "epa")
epa_stats["def_past40_total_epa"], epa_stats["def_past40_epa_per_play"] = calc_epa_stats(p40_df, "def_pos_team_id", "def_epa")

# Penalty EPA
penalty_df = plays_df[penalty_filter]
epa_stats["total_penalty_epa"] = penalty_df.groupby("pos_team_id")["epa"].sum()
epa_stats["def_total_penalty_epa"] = penalty_df.groupby("def_pos_team_id")["def_epa"].sum()

# Timeout EPA
timeout_df = plays_df[timeout_filter]
epa_stats["timeout_total_epa"] = timeout_df.groupby("pos_team_id")["epa"].sum()
epa_stats["def_timeout_total_epa"] = timeout_df.groupby("def_pos_team_id")["def_epa"].sum()

# Finalize: reset index and handle missing values
epa_stats = epa_stats.reset_index().rename(columns={"pos_team_id": "team_id"})
epa_stats["team_id"] = epa_stats["team_id"].astype(int)
epa_stats = epa_stats.fillna(0.0)

In [166]:
def calc_success_rate(df, team_col, epa_col, condition=None):
    if condition is not None:
        filtered = df[condition]
    else:
        filtered = df

    total_plays = filtered[team_col].value_counts().sort_index()
    success_plays = filtered[filtered[epa_col] > 0][team_col].value_counts().sort_index()

    success_plays = success_plays.reindex(total_plays.index, fill_value=0)
    success_rate = success_plays / total_plays
    return success_rate

# Initialize results dictionary
success_rates = {}

# Overall success rate
success_rates["success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa")
success_rates["def_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa")

# Passing success rate
pass_cond = plays_df["pass"] == 1
success_rates["passing_success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa", condition=pass_cond)
success_rates["def_passing_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa", condition=pass_cond)

# Rushing success rate
rush_cond = plays_df["rush"] == 1
success_rates["rushing_success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa", condition=rush_cond)
success_rates["def_rushing_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa", condition=rush_cond)

# Down-based success
for down_num in range(1, 5):
    cond = plays_df["down"] == down_num
    label = ["first", "second", "third", "fourth"][down_num - 1]
    success_rates[f"{label}down_success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa", condition=cond)
    success_rates[f"def_{label}down_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa", condition=cond)

# Red zone (yards_to_goal <= 20)
rz_cond = plays_df["yards_to_goal"] <= 20
success_rates["redzone_success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa", condition=rz_cond)
success_rates["def_redzone_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa", condition=rz_cond)

# Past 40 (yards_to_goal <= 40)
p40_cond = plays_df["yards_to_goal"] <= 40
success_rates["past40_success_rate"] = calc_success_rate(plays_df, "pos_team_id", "epa", condition=p40_cond)
success_rates["def_past40_success_rate"] = calc_success_rate(plays_df, "def_pos_team_id", "def_epa", condition=p40_cond)

# Convert to DataFrame
success_df = pd.DataFrame(success_rates)

# Reset index so team_id is a column like in epa_stats
success_df = success_df.reset_index().rename(columns={"index": "team_id"})
success_df["team_id"] = success_df["team_id"].astype(int)

# Fill missing values
success_df = success_df.fillna(0.0)

In [167]:
print(success_df)

     team_id  success_rate  def_success_rate  passing_success_rate  \
0          0           0.0          0.479592                   0.0   
1          1       0.38789          0.516035              0.340136   
2          2      0.382269          0.528263              0.354626   
3          3      0.415146          0.571304              0.457672   
4          4           0.0          0.431373                   0.0   
..       ...           ...               ...                   ...   
223      292           0.0          0.494624                   0.0   
224      294      0.408437          0.551648              0.401596   
225      295           1.0           0.43956                   0.0   
226      296      0.285714          0.528485                   0.0   
227      298           0.0          0.366197                   0.0   

     def_passing_success_rate  rushing_success_rate  def_rushing_success_rate  \
0                    0.404762                   0.0                  0.516129 

# Insert Data into Tables

In [168]:
from psycopg2.extras import execute_values

def insert_dataframe(cursor, conn, df, table_name, columns, pk):
    # Clear any failed transaction
    try:
        conn.rollback()
    except:
        pass
    
    cleaned_df = df[columns].copy()

    # Convert nullable Int64/string columns to object dtype (avoids psycopg2 issues)
    for col in cleaned_df.columns:
        if pd.api.types.is_integer_dtype(cleaned_df[col]) or pd.api.types.is_string_dtype(cleaned_df[col]):
            cleaned_df[col] = cleaned_df[col].astype(object)

    # Convert all pd.NA / np.nan / NaT to None
    cleaned_df = cleaned_df.where(pd.notnull(cleaned_df), None) # Convert NaN to None
    values = [tuple(x) for x in cleaned_df.to_numpy()]
    
    updates = ', '.join([f"{col}=EXCLUDED.{col}" for col in columns if col != pk])
    
    insert_sql = f"""
        INSERT INTO {table_name} ({', '.join(columns)}) 
        VALUES %s 
        ON CONFLICT ({pk}) DO UPDATE SET {updates}
    """
    
    execute_values(cursor, insert_sql, values)
    conn.commit()

# 1. Insert into teams (top-most table, referenced by many others)
insert_dataframe(cur, conn, teams_df, "teams", ["team_id", "team_name", "conference", "city", "state"], pk="team_id")

# 2. Insert into games (needs teams to exist first)
insert_dataframe(cur, conn, games_df, "games", [
    "game_id", "year", "week", "home_team_id", "home", "away_team_id", "away", "home_score", "away_score"
], pk="game_id")

# 3. Insert into drives (references games and teams)
insert_dataframe(cur, conn, drives_df, "drives", [
    "drive_id", "game_id", "drive_num", "pos_team_id", "pos_team", "drive_result",
    "drive_yards", "drive_start_yards_to_goal", "drive_end_yards_to_goal", "drive_points",
    "drive_time_minutes_start", "drive_time_seconds_start", "drive_time_minutes_end",
    "drive_time_seconds_end", "drive_time_minutes_elapsed", "drive_time_seconds_elapsed"
], pk="drive_id")

# 4. Insert into plays (depends on drives, games, and teams)
insert_dataframe(cur, conn, plays_df, "plays", [
    "id_play", "game_id", "game_play_number", "drive_id", "half", "period",
    "clock_minutes", "clock_seconds", "time_secs_rem",
    "pos_team", "pos_team_id", "def_pos_team", "def_pos_team_id",
    "play_type", "play_text", "pass", "pass_attempt", "completion", "rush",
    "yards_gained", "down", "distance", "yards_to_goal",
    "epa", "def_epa", "wpa", "ep_before", "ep_after", "wp_before", "wp_after",
    "sack", "int", "penalty_detail", "yds_penalty", "firstd_by_penalty",
    "firstd_by_yards", "success", "turnover", "turnover_downs", "score_pts",
    "scoring_play", "defense_score_play", "change_of_poss", "off_timeout",
    "off_timeouts_rem_before", "def_timeout", "def_timeouts_rem_before"
], pk="id_play")

# 5–10. Insert summary/statistical tables (only depend on teams)

insert_dataframe(cur, conn, epa_stats, "epa", [
    "team_id", "epa_per_play", "def_epa_per_play", "total_epa", "def_total_epa",
    "passing_epa_per_play", "def_passing_epa_per_play", "passing_total_epa", "def_passing_epa_total",
    "rushing_epa_per_play", "def_rushing_epa_per_play", "rushing_total_epa", "def_rushing_total_epa",
    "firstdown_epa_per_play", "def_firstdown_epa_per_play", "seconddown_epa_per_play", "def_seconddown_epa_per_play",
    "thirddown_epa_per_play", "def_thirddown_epa_per_play", "fourthdown_epa_per_play", "def_fourthdown_epa_per_play",
    "redzone_epa_per_play", "def_redzone_epa_per_play", "redzone_total_epa", "def_redzone_total_epa",
    "past40_epa_per_play", "def_past40_epa_per_play", "past40_total_epa", "def_past40_total_epa",
    "total_penalty_epa", "def_total_penalty_epa", "timeout_total_epa", "def_timeout_total_epa"
], pk="team_id")

insert_dataframe(cur, conn, success_df, "success", [
    "team_id", "success_rate", "def_success_rate", "passing_success_rate", "def_passing_success_rate",
    "rushing_success_rate", "def_rushing_success_rate", "firstdown_success_rate", "def_firstdown_success_rate",
    "seconddown_success_rate", "def_seconddown_success_rate", "thirddown_success_rate", "def_thirddown_success_rate",
    "fourthdown_success_rate", "def_fourthdown_success_rate", "redzone_success_rate", "def_redzone_success_rate",
    "past40_success_rate", "def_past40_success_rate"
], pk="team_id")

insert_dataframe(cur, conn, pff_passing_df, "pff_passing", [
    "team_id", "team_name", "attempts", "avg_depth_of_target", "avg_time_to_throw", "big_time_throws",
    "completion_percentage", "completions", "def_gen_pressures", "dropbacks", "drops", "grade_offense",
    "grade_passing", "grade_run", "hit_as_threw", "interceptions", "sacks", "scrambles", "thrown_away",
    "touchdowns", "turnover_worthy_plays", "yards"
], pk="team_id")

insert_dataframe(cur, conn, pff_rushing_df, "pff_rushing", [
    "team_id", "team_name", "attempts", "avoided_tackles", "breakaway_yards", "first_downs", "fumbles",
    "grades_offense", "grades_pass_block", "grades_run", "rec_yards", "receptions", "routes", "scramble_yards",
    "targets", "touchdowns", "yards", "yards_after_contact"
], pk="team_id")

insert_dataframe(cur, conn, pff_receiving_df, "pff_receiving", [
    "team_id", "team_name", "contested_receptions", "contested_targets", "grades_offense", "grades_pass_route",
    "receptions", "routes", "yards", "yards_after_catch"
], pk="team_id")

insert_dataframe(cur, conn, pff_blocking_df, "pff_blocking", [
    "team_id", "team_name", "grades_offense", "grades_pass_block", "grades_run_block", "hits_allowed",
    "hurries_allowed", "penalties", "pressures_allowed", "sacks_allowed"
], pk="team_id")

insert_dataframe(cur, conn, pff_defense_df, "pff_defense", [
    "team_id", "team_name", "batted_passes", "forced_fumbles", "fumble_recoveries", "grades_coverage_defense",
    "grades_defense", "grades_pass_rush_defense", "grades_run_defense", "grades_tackle", "hits", "hurries",
    "interceptions", "missed_tackles", "pass_break_ups", "penalties", "qb_rating_against", "receptions",
    "sacks", "safeties", "stops", "tackles", "tackles_for_loss", "total_pressures", "touchdowns",
    "yards", "yards_after_catch"
], pk="team_id")

CardinalityViolation: ON CONFLICT DO UPDATE command cannot affect row a second time
HINT:  Ensure that no rows proposed for insertion within the same command have duplicate constrained values.
